In [1]:
from pathlib import Path
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np

os.environ["WANDB_SILENT"] = "true"

from pathlib import Path
import numpy as np
import pandas as pd

from owkin.dataset import build_dataset

In [2]:
data_path = "../data/"
"""
Build `X_train_mean, X_val_mean, X_train, X_val, y_train, y_val`
TODO: add other aggregation that mean (e.g. max) ? Use a dict ?
"""

# put your own path to the data root directory (see example in `Data architecture` section)
data_dir = Path(data_path)

# load the training and testing data sets
train_features_dir = data_dir / "train_input" / "moco_features"
test_features_dir = data_dir / "test_input" / "moco_features"
df_train_val = pd.read_csv(data_dir / "supplementary_data" / "train_metadata.csv")
df_test = pd.read_csv(data_dir / "supplementary_data" / "test_metadata.csv")


# concatenate y_train_val and df_train_val
y_train_val = pd.read_csv(data_dir / "train_output.csv")
df_train_val = df_train_val.merge(y_train_val, on="Sample ID")


In [3]:
# Path for normalization factor
data_norm_path = Path('../data/expo_lambda')

center_list_train = ['C_1', 'C_2', 'C_5']
center_list_val = ['C_3', 'C_4']

X_dict = {}
y_dict = {}

# Extract and normalize
for center_curr in center_list_train:
    X_train = []
    y_train = []

    for sample, label, center, patient in df_train_val[
        ["Sample ID", "Target", "Center ID", "Patient ID"]
    ].values:
        if center == center_curr:
            # load the coordinates and features (1000, 3+2048)
            _features = np.load(train_features_dir / sample)
            # get coordinates (zoom level, tile x-coord on the slide, tile y-coord on the slide)
            # and the MoCo V2 features
            coordinates, features = _features[:, :3], _features[:, 3:]  # Ks
            X_train.append(features)
            y_train.append(label)

    # convert to numpy arrays
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    # normalize
    norm_factor = np.load(data_norm_path / f'{center_curr}.npy')
    X_train = X_train / norm_factor

    X_dict[center_curr] = X_train
    y_dict[center_curr] = y_train

In [6]:
center_list_val = ['C_3', 'C_4']

# Extract and normalize
for center_curr in center_list_val:
    X_train = []
    y_train = []

    for sample, label, center, patient in df_test[
        ["Sample ID", "Target", "Center ID", "Patient ID"]
    ].values:
        if center == center_curr:
            # load the coordinates and features (1000, 3+2048)
            _features = np.load(test_features_dir / sample)
            # get coordinates (zoom level, tile x-coord on the slide, tile y-coord on the slide)
            # and the MoCo V2 features
            coordinates, features = _features[:, :3], _features[:, 3:]  # Ks
            X_train.append(features)
            y_train.append(label)

    # convert to numpy arrays
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    # normalize
    norm_factor = np.load(data_norm_path / f'{center_curr}.npy')
    X_train = X_train / norm_factor

    X_dict[center_curr] = X_train
    y_dict[center_curr] = y_train

KeyError: "['Target'] not in index"